In [1]:
import sys
import os 
sys.path.insert(1, os.path.realpath(os.path.pardir))

import numpy as np
from pathlib import Path

import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from utils import data_utils
from utils import losses, hand_visualize

from models import HVATNet_v2


## Data preprocessing

In [2]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [3]:
config = dict(
    WANDB_NOTES = 'HVATNet v3(simple) Matvey + ds 8',
    # data = '/home/user/common/koval_alvi/myo_to_hand_pose/data/anna_12_08',
    # data = '/home/user/common/koval_alvi/myo_to_hand_pose/data/experiment_v2/study_amp_Andrey_05_11_22',
    # data = '/home/user/common/koval_alvi/myo_to_hand_pose/data/experiment_v2/study_health_Anna_20_12_22',
    data = '/home/user/common/koval_alvi/myo_to_hand_pose/data/experiment_v2/health_right_matvey_26_01_23',


    original_fps = 200,  # TODO describtion
    delay_ms = 0,  # Shift vr vs EMG parameter. ms dealy between emg and VR.
    start_crop_ms = 5*1000,  # bad values in the beginning of recordign in ms to delete.
    window_size = 512,

    max_epochs = 500,
    samples_per_epoch = 100000 ,#160000, 
    train_bs = 256,
    val_bs = 32,
    device = [3],
    optimizer_params = dict(lr=1e-4,
                            wd=1e-5)
)
config = AttrDict(config)

In [4]:
import audiomentations as A
augment = A.Compose([
    # A.Gain(min_gain_in_db=-0.05, max_gain_in_db=0.05, p=0.3),
    # A.GainTransition(
    #     min_gain_in_db = -0.5,
    #     max_gain_in_db =  0.5,
    #     min_duration = 10,
    #     max_duration = 100,
    #     duration_unit = "samples",
    #     p=0.3),
    A.AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.2, p=0.5),
    # data_utils.SpatialRotation(min_angle=1, max_angle=10, p=0.5)
])

x = torch.zeros(8, 512)
augmented_samples = augment(samples=x, sample_rate=200) # sahpe -> [8, time]
print(augmented_samples.shape)

(8, 512)


In [5]:
# Init train and val dataset and dataloaders
all_data_path = Path(config.data)
train_folder = all_data_path / 'train'
val_folder = all_data_path / 'test'

train_dataset = data_utils.create_dataset(data_folder=train_folder,
                                          original_fps=config.original_fps,
                                          delay_ms=config.delay_ms,
                                          start_crop_ms=config.start_crop_ms,
                                          window_size=config.window_size,
                                          random_sampling=True,
                                          samples_per_epoch=config.samples_per_epoch, 
                                          transform = augment, 
                                          down_sample_target=8)

val_dataset = data_utils.create_dataset(data_folder=val_folder,
                                        original_fps=config.original_fps,
                                        delay_ms=config.delay_ms,
                                        start_crop_ms=config.start_crop_ms,
                                        window_size=config.window_size,
                                        random_sampling=False,
                                        samples_per_epoch=None, 
                                        transform=None)



Number of all files with moves: 58


100%|██████████| 58/58 [02:46<00:00,  2.87s/it]


Total len: 100000
Number of all files with moves: 15


100%|██████████| 15/15 [00:42<00:00,  2.86s/it]

Total len: 307


# Init model

In [6]:
class LitHVATNet_v2(pl.LightningModule):
    def __init__(self, model, loss_function, lr, wd):
        """
        Wrapper of model with loss function calculatino and initing optimizer.
        """
        super().__init__()
        self.model = model
        self.lr = lr
        self.wd = wd
        self.loss_function = loss_function # should compare quats. 
    
    def forward(self, x):
        x = self.model(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), 
                                      lr=self.lr, 
                                      weight_decay = self.wd)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        multi_scale_quats = self.model(x)
        
        # loss on full size
        full_size_pred = multi_scale_quats[-1]
        loss_dict = self.loss_function(full_size_pred, y)
        
        # average loss on multi scale outputs
        # for interpolation purpose 
        batch, n_bones, n_quats, time = y.shape
        y_3d = y.reshape(batch, -1, time)
        
        ms_losses = []
        for feat in multi_scale_quats[:-1]:  
            
            new_time_size = feat.shape[-1]
            y_ = F.interpolate(y_3d, size=new_time_size, mode='linear', align_corners=False)
            y_ = y_.reshape(batch, n_bones, n_quats, new_time_size)
            
            loss_dict_tmp = self.loss_function(feat, y_)
            ms_losses.append(loss_dict_tmp['total_loss'])
        
        ms_loss = torch.mean(torch.stack(ms_losses))
        loss_dict['total_loss'] += ms_loss
        
        for k, v in loss_dict.items():
            self.log("train_" + str(k), v, on_step=False, on_epoch=True)
        return loss_dict['total_loss']
            
    def validation_step(self, val_batch, batch_idx):
        
        if trainer.global_step == 0: 
            wandb.define_metric('val_angle_degree', summary='min')
        
        x, y = val_batch
        full_size_pred = self.model(x)
        loss_dict = self.loss_function(full_size_pred, y)
                
        for k, v in loss_dict.items():
            self.log("val_" + str(k), v, on_step=False, on_epoch=True)
        return loss_dict['angle_degree']
    
    
    def validation_epoch_end(self, validation_step_outputs):
        val_current_loss = torch.mean(torch.stack(validation_step_outputs))
        self.val_current_loss = val_current_loss
        print(f'current step {self.current_epoch} val_current_loss {self.val_current_loss}')

In [7]:
from pytorch_lightning.callbacks import Callback

class MovementsWandb(Callback):
    def __init__(self):
        self.best_val_loss = 10000000
        
    def on_validation_epoch_end(self, trainer, pl_module):
        
        ### do not calculate epoch in sanity check. 
        if trainer.state.stage == 'sanity_check':
            return

        # check better or not
        if pl_module.val_current_loss < self.best_val_loss: 
            
            print('Best val')
            print('MAKE VIS', pl_module.current_epoch)
            print('new best val score', pl_module.val_current_loss) 
            
            self.best_val_loss = pl_module.val_current_loss
            
            hand_visualize.visualize_val_moves(model = pl_module.model, 
                                               val_exps_data = trainer.val_dataloaders[0].dataset.exps_data, 
                                               epoch = pl_module.current_epoch,
                                               device = pl_module.device)
                

                

## Init model

In [8]:
hvatnet_v2_params =dict(n_electrodes=8, n_channels_out=64,
                        n_res_blocks=3, n_blocks_per_layer=2,
                        n_filters=128, kernel_size=3,
                        strides=(2, 2, 2, 4),
                        dilation=2)

model = HVATNet_v2.HVATNetv2(**hvatnet_v2_params)
model_pl = LitHVATNet_v2(model,
                       loss_function=losses.make_loss_function(),
                       **config.optimizer_params)


### Start to train model.

In [ ]:

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=config.train_bs,
                                               shuffle=True,
                                               num_workers=0)

val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=config.val_bs,
                                             shuffle=False,
                                             num_workers=0)


    
    
wandb_logger = WandbLogger(entity="koval_alvi",
                           project="ALVI_hvatnet_v2", 
                           log_model = 'all', 
                           save_code=True,
                           notes = config.WANDB_NOTES, 
                           dir = 'lightning_logs')

checkpoint_callback = ModelCheckpoint(monitor='val_angle_degree',
                                  save_top_k=5,
                                  save_last=True,
                                  filename='{epoch:02d}-{val_angle_degree:.3f}',
                                  verbose=True,
                                  mode='min')

trainer = pl.Trainer(max_epochs=config.max_epochs,
                     accelerator='gpu', 
                     devices= config.device,
                     logger=wandb_logger,
                     callbacks=[MovementsWandb(), checkpoint_callback])

trainer.fit(model_pl, train_dataloader, val_dataloader)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type      | Params
------------------------------------
0 | model | HVATNetv2 | 3.6 M 
------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.431    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


current step 0 val_current_loss 125.7862548828125


...

Epoch 192, global step 75463: 'val_angle_degree' was not in top 5


current step 192 val_current_loss 10.043913841247559


Validation: 0it [00:00, ?it/s]

Epoch 193, global step 75854: 'val_angle_degree' was not in top 5


current step 193 val_current_loss 10.208563804626465


### Inference and model uploading

In [ ]:
# reference can be retrieved in artifacts panel
# "VERSION" can be a version (ex: "v2") or an alias ("latest or "best")
checkpoint_reference = "koval_alvi/alvi labs hand prediction/model-1wsrrin6:v19"

# download checkpoint locally (if not already cached)
run = wandb.init(project="alvi labs hand prediction")
artifact = run.use_artifact(checkpoint_reference, type="model")
artifact_dir = artifact.download()

# load checkpoint
model = model.load_state_dict(torch.load(Path(artifact_dir) / "model.ckpt"))

In [ ]:
weights = torch.load(Path(artifact_dir) / "model.ckpt")['state_dict']

model_pl.load_state_dict(weights)

In [ ]:
model_pl